In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [13]:

df = pd.read_csv('Data/Train.csv')
df.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png


In [14]:
train_path = 'Train'

In [16]:
from tensorflow.keras.applications.vgg16 import preprocess_input

image_generator = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)


In [17]:
train_batches = image_generator.flow_from_directory(
    directory=train_path,
    target_size=(224, 224),
    batch_size=20,
    subset='training',
    class_mode='categorical'
)

val_batches = image_generator.flow_from_directory(
    directory=train_path,
    target_size=(224, 224),
    batch_size=20,
    subset='validation',
    class_mode='categorical'
)

Found 31368 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


In [18]:
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [19]:
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(43, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=x)

In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Set up model checkpoint to save the best model during training
checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')

# Train the model
history = model.fit(
    train_batches,
    epochs=20,  # adjust as needed
    validation_data=val_batches,
    callbacks=[checkpoint]
)

In [ ]:
# Plot accuracy and loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.show()